In [ ]:
"""
This is a program can compute correlations within data
"""

In [1]:
# load packages
import numpy as np
import pandas as pd
import seaborn as sns
import random
from matplotlib import pyplot as plt

"""
 p defines the percentage of the lines
 if random from [0,1] interval is greater than 0.01 the row will be skipped
 originally from stackoverflow question no.22258491
"""

p = 0.1
train = pd.read_csv('SharedResponsesFullFirstSessions.csv'
                    ,skiprows = lambda x: x >  0 and random.random() > p)


In [3]:
# check how they match up
train['Combo'] = train['ScenarioType'] +' '+ train['DefaultChoice']
print("Possiable combo for 'ScenarioType'&'DefaultChoice' could be:")
print(train['Combo'].unique())

Possiable combo for 'ScenarioType'&'DefaultChoice' could be:
['Utilitarian More' 'Age Young' 'Fitness Fit' nan 'Species Hoomans'
 'Gender Male' 'Social Status High']


In [4]:
# remove some irrelavent columns
drop_columns = ['UserID','ScenarioOrder','ResponseID','ExtendedSessionID',
             'DescriptionShown','LeftHand','Template'] 
print("Dropped:",drop_columns)

train.drop(drop_columns,axis=1, inplace=True)

Dropped: ['UserID', 'ScenarioOrder', 'ResponseID', 'ExtendedSessionID', 'DescriptionShown', 'LeftHand', 'Template']


In [12]:
# removing randomness
mask1 = train['ScenarioTypeStrict'] != 'Random'
mask2 = train['ScenarioType'] != 'Random'
mask3 = pd.concat((mask1,mask2),axis= 1)
mask4 = mask3.all(axis=1)

train = train[mask4]

In [13]:
# if there is anything that is (highly) relevant for DefaultChoiceIsOmission
# if all 'AttributeLevel' & 'DefaultChoice' are the same

train['Flipped'] = (train['AttributeLevel'] == train['DefaultChoice'])
average = (train['Flipped'] == train['DefaultChoiceIsOmission'] ).mean()
print(train['AttributeLevel'].unique())
print(train['DefaultChoice'].unique())
print(train['DefaultChoiceIsOmission'].unique())
print(train['Flipped'].unique())

"""
0      : they are opposite to each other
0<avg<1: in between
1      : they are the same

"""

print(100*average)

# see the pairs:
# train[['Flipped','DefaultChoiceIsOmission']]

['More' 'Young' 'Fit' 'Pets' 'Fat' 'Less' 'Old' 'Hoomans' 'Female' 'Male'
 'High' 'Low']
['More' 'Young' 'Fit' 'Hoomans' 'Male' 'High']
[1. 0.]
[ True False]
50.10446902835199


In [18]:
# if Intervention is somehow relevant

train['FlippedAgain'] = (train['DefaultChoiceIsOmission'] == train['Intervention'])
average = (train['Flipped'] == train['FlippedAgain'] ).mean()
print(train['DefaultChoiceIsOmission'].unique())
print(train['Intervention'].unique())
print(train['FlippedAgain'].unique())
print(100*average)

# see the pairs:
# train[['Flipped','FlippedAgain']]

[1. 0.]
[0 1]
[False  True]
0.0


In [16]:
# generate the correlation map

train['AttributeLevel'],_ = train['AttributeLevel'].factorize()
train['DefaultChoice'],_  = pd.factorize(train['DefaultChoice'])
train['ScenarioType'],_   = pd.factorize(train['ScenarioType'])

corr = train.corr()

In [ ]:
# display the correlation

print("Correlation for variables:")

plt.figure(figsize=(11,9))
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    annot=True,fmt=".3f",
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)